# Generate a new complex group label for a spreadsheet

## Criteria

Noncarriers
__group 1: noncarriers__ 
- FAMILY_GENE = either C9, MAPT or GRN
- GENETIC_STATUS = NONE
- CLIN_STATUS = 0 (*Important* that they stay 0 through all of their visits)
- earliest visit only ('VIST_NUM'==1)


__group 2: FTLDCDR0<40__
- GENETIC_STATUS = either  C9, MAPT, GRN or 'C9 and GRN'
- CLIN_STATUS = 0 (*Important* that they stay 0 through all of their visits)
- AGE = <40 years
- earliest visit only ('VIST_NUM'==1)

__group 3: FTLDCDR0>=40__
 - GENETIC_STATUS = either  C9, MAPT, GRN or 'C9 and GRN'
 - CLIN_STATUS = 0 (*Important* that they stay 0 through all of their visits)
 - AGE = >=40 years
 - earliest visit only ('VIST_NUM'==1)

__group 4: FTLDCDR0.5__
- CLIN_STATUS = 0.5
- can become 1, or 2, or 3, or 0
- earliest visit they were 0.5

__group 5: FTLDCDR1__
- CLIN_STATUS = 1 !but were never 0.5
- they can become 2 or 3.
- earlist visit they are 1

__A critical part of this is that we are create a new column, rather than a new dataframe altogether.__

To make this work we will be using the dataframe index. This is a extra column that pandas creates that records every row of the df - much like how excel has a number of each row. The important part is that it keeps a note of these even when you manipulate a df.

In [1]:
import pandas as pd
import numpy as np
spreadsheet_file = 'example_data.xlsx'
df = pd.read_excel(spreadsheet_file)

In [2]:
# our aim to to populate this new column with the correct group labels
df['group_label'] = 'NoGroup'

## Group 1

In [3]:
new_df = df.loc[(df['FAMILY_GENE'] =='C9') | (df['FAMILY_GENE'] == 'MAPT') |  (df['FAMILY_GENE']=='GRN')]
new_df = new_df.loc[new_df['GENETIC_STATUS']=='NONE']

# let's copy this to a dataframe called group1_df
group1_df = new_df.copy()
display(group1_df)

,ID,VIST_NUM,CLIN_STATUS,FAMILY_GENE,GENETIC_STATUS,AGE,SEX,COG_MEM,COG_ORI,COG_JUDG,COG_LANG,COG_VIS,COG_ATTN,group_label
8,203002843,1,0.0,GRN,NONE,35,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
14,203005860,1,0.0,GRN,NONE,54,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
15,203005860,2,0.0,GRN,NONE,55,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
16,203005860,3,0.0,GRN,NONE,56,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
17,203005860,4,0.0,GRN,NONE,58,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,203452361,1,0.0,MAPT,NONE,52,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
943,203452361,2,0.0,MAPT,NONE,53,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
944,203452361,3,0.0,MAPT,NONE,55,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup
951,203458650,1,0.5,GRN,NONE,60,1,1.0,0.0,0.0,0.0,0.0,0.0,NoGroup


Notice that when we display the group1_df the first row is actually '8'? It has saved the original row numbers from 'df'. Most of the time you can ignore, or even reset this information but in this case we will use it later.

## Group 2

In [4]:
new_df = df.loc[(df['GENETIC_STATUS'] =='C9') | (df['GENETIC_STATUS'] == 'MAPT') |  (df['GENETIC_STATUS']=='GRN') | (df['GENETIC_STATUS']=='C9 and GRN')]
new_df = new_df.loc[new_df['AGE'] < 40]
group2_df = new_df.copy()

## Group 3

In [5]:
new_df = df.loc[(df['GENETIC_STATUS'] =='C9') | (df['GENETIC_STATUS'] == 'MAPT') |  (df['GENETIC_STATUS']=='GRN') | (df['GENETIC_STATUS']=='C9 and GRN')]
new_df = new_df.loc[new_df['AGE'] >= 40]
group3_df = new_df.copy()

### Extra rules that apply to the first three groups

In [6]:
group1_df['group'] = 'noncarriers'
group2_df['group'] = '< 40'
group3_df['group'] = '0>=40'
groups_df = pd.concat([group1_df,group2_df,group3_df])

## First rule
groups_df['index'] = 0
for subject_id in groups_df['ID'].unique():
    
    # define a subset df with just the subject information
    subj_df = groups_df.loc[groups_df['ID']==subject_id]
    
    # IF all the GLOB measures = 0
    if all(subj_df['CLIN_STATUS']==0):
        # add '1s' to a variable we can use to filter the df later
        groups_df.loc[groups_df['ID']==subject_id,'index'] = 1
        
# only keep the Glob_index rows
groups_df = groups_df.loc[groups_df['index']==1]

## Second rule
groups_df = groups_df.loc[groups_df['VIST_NUM']==1]

## Display
display(groups_df)

,ID,VIST_NUM,CLIN_STATUS,FAMILY_GENE,GENETIC_STATUS,AGE,SEX,COG_MEM,COG_ORI,COG_JUDG,COG_LANG,COG_VIS,COG_ATTN,group_label,group,index
8,203002843,1,0.0,GRN,NONE,35,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,noncarriers,1
14,203005860,1,0.0,GRN,NONE,54,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,noncarriers,1
28,203011157,1,0.0,GRN,NONE,74,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,noncarriers,1
36,203013437,1,0.0,MAPT,NONE,36,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,noncarriers,1
68,203027054,1,0.0,C9,NONE,36,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,noncarriers,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,203455123,1,0.0,MAPT,MAPT,40,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,0>=40,1
963,203463601,1,0.0,GRN,GRN,43,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,0>=40,1
971,203467991,1,0.0,C9,C9,47,2,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,0>=40,1
974,203468453,1,0.0,NONE,GRN,59,1,NaN,NaN,NaN,NaN,NaN,NaN,NoGroup,0>=40,1


In [7]:
#record the indices for the groups
group1_index = groups_df.loc[groups_df['group']=='noncarriers'].index
group2_index = groups_df.loc[groups_df['group']=='< 40'].index
group3_index = groups_df.loc[groups_df['group']=='0>=40'].index

## Group 4
__group 4: FTLDCDR0.5__
- CLIN_STATUS = 0.5
- can become 1, or 2, or 3, or 0
- earliest visit they were 0.5

In [8]:
group4_df = df.loc[df['CLIN_STATUS'] ==0.5]
group4_index = []

for subject_id in group4_df['ID'].unique():
    
    # define a subset df with just the subject information
    subj_df = group4_df.loc[group4_df['ID']==subject_id]
    
    # when was the first visit (with 0.5)?
    first_visit = min(subj_df['VIST_NUM'])
    group4_index.append(subj_df.loc[subj_df['VIST_NUM']==first_visit].index[0])

In [9]:
group1_index

Int64Index([  8,  14,  28,  36,  68,  88,  91, 150, 162, 166, 174, 191, 199,
            213, 230, 245, 262, 268, 270, 275, 309, 318, 343, 365, 373, 382,
            386, 405, 418, 421, 430, 439, 445, 451, 460, 464, 470, 494, 512,
            520, 523, 534, 552, 580, 583, 591, 595, 598, 617, 623, 632, 657,
            659, 687, 690, 696, 714, 722, 746, 750, 751, 763, 767, 773, 838,
            855, 858, 860, 863, 885, 889, 905, 909, 919, 929, 934, 942],
           dtype='int64')

## Group 5
__group 5: FTLDCDR1__
- CLIN_STATUS = 1 !but were never 0.5
- they can become 2 or 3.
- earlist visit they are 1

In [10]:
group5_df = df.loc[df['CLIN_STATUS'] ==1]
group5_index = []

for subject_id in group5_df['ID'].unique():
    
    # define a subset df with just the subject information
    subj_df = group5_df.loc[group5_df['ID']==subject_id]
    
    if not any(subj_df['CLIN_STATUS']==0.5):
        # when was the first visit (with 1.0)?
        first_visit = min(subj_df['VIST_NUM'])
        group5_index.append(subj_df.loc[subj_df['VIST_NUM']==first_visit].index[0])

# Put all the indicies together

In [12]:
df.loc[group1_index,'group_label'] = 'noncarriers'
df.loc[group2_index,'group_label'] = '<40'
df.loc[group3_index,'group_label'] = '>=40'
df.loc[group4_index,'group_label'] = '0.5'
df.loc[group5_index,'group_label'] = '1.0'
df.groupby('group_label').mean()
#df.to_excel('grouped_data.xlsx')

,ID,VIST_NUM,CLIN_STATUS,AGE,SEX,COG_MEM,COG_ORI,COG_JUDG,COG_LANG,COG_VIS,COG_ATTN
group_label,,,,,,,,,,,
0.5,2.032194e+08,1.174603,0.500000,62.738095,1.515873,0.408163,0.081633,0.632653,0.653061,0.081633,0.336735
1.0,2.032316e+08,1.076923,1.000000,65.828402,1.437870,0.730539,0.353293,0.928144,0.706587,0.215569,0.658683
<40,2.032146e+08,1.000000,0.000000,31.677419,1.709677,NaN,NaN,NaN,NaN,NaN,NaN
>=40,2.032472e+08,1.000000,0.000000,54.285714,1.571429,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
NoGroup,2.032350e+08,1.873214,0.898214,54.694643,1.548214,0.891566,0.658635,0.979920,0.734940,0.634538,0.815261
noncarriers,2.032355e+08,1.000000,0.000000,46.922078,1.584416,NaN,NaN,NaN,NaN,NaN,NaN
